
## **Motion Detection and Tracking**

**Motion Detection and Video Recording System Using Frame Processing in OpenCV**

In [1]:
import cv2
import numpy as np

In [2]:
def preprocess_frame(frame: np.ndarray, ksize: tuple = (7, 7)) -> np.ndarray:
    """
    Converts a frame to grayscale and applies Gaussian blur.
        return processed_frame
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, ksize, 0)
    return blurred_frame


def get_video_properties(video_capture: cv2.VideoCapture) -> tuple:
    """
    Retrieves video properties like frame width, height, and FPS.
        return frame_width, frame_height, fps
    """
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    return frame_width, frame_height, fps


def initialize_video_writer(
    output_path: str, frameSize: tuple, fps: int, codec: str = "XVID"
) -> cv2.VideoWriter:
    """
    Initializes the VideoWriter object for saving the video.
        return video_writer
    """
    fourcc = cv2.VideoWriter_fourcc(*codec)  # Codec for the video file
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, frameSize)
    return video_writer

In [3]:
cap = cv2.VideoCapture("../Data/Videos/People_Moving.avi")
frame_width, frame_height, fps = get_video_properties(cap)

output_path = "../Data/Videos/Detected_Motion.avi"
video_writer = initialize_video_writer(output_path, (frame_width, frame_height), fps)

# Read the first frame and preprocess it
ret, initial_frame = cap.read()
processed_initial_frame = preprocess_frame(initial_frame)

while True:
    # Read the current frame
    ret, current_frame = cap.read()
    if not ret:
        break

    processed_current_frame = preprocess_frame(current_frame)

    frame_difference = cv2.absdiff(processed_initial_frame, processed_current_frame)
    _, binary_threshold = cv2.threshold(frame_difference, 25, 255, cv2.THRESH_BINARY)
    dilated_image = cv2.dilate(binary_threshold, None, iterations=3)

    contours, _ = cv2.findContours(
        dilated_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Draw rectangles around detected motion
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Ignore small movements
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(current_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    video_writer.write(current_frame)

    cv2.imshow("Threshold", binary_threshold)
    cv2.imshow("After Dilation", dilated_image)
    cv2.imshow("Motion Detection", current_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    # Update the reference frame dynamically for continuous motion tracking
    processed_initial_frame = processed_current_frame


cap.release()
video_writer.release()
cv2.destroyAllWindows()

1. Read Frame
2. Preprocess Frame
3. Apply Function
4. Threshold
5. Dilation
6. Find Contours